In [10]:
from gurobipy import *
import random as rd
import json

### Looking at the data

In [11]:
# Opening JSON file
f = open('/Users/cha/Desktop/Code/optimization-project/data/medium.json')
data = json.load(f)
data.keys()
f.close()

In [12]:
data['horizon'], data['qualifications']

(22, ['F', 'G', 'A', 'D', 'J', 'C', 'H', 'E', 'B', 'I'])

In [13]:
data['staff']

[{'name': 'Olivia', 'qualifications': ['A', 'B', 'C'], 'vacations': [1, 2]},
 {'name': 'Liam', 'qualifications': ['A', 'D', 'E'], 'vacations': [1, 2]},
 {'name': 'Emma', 'qualifications': ['B', 'H'], 'vacations': [8, 9]},
 {'name': 'Noah',
  'qualifications': ['G', 'D', 'J', 'C', 'H', 'I'],
  'vacations': []},
 {'name': 'Amelia',
  'qualifications': ['F', 'G', 'J', 'E'],
  'vacations': [16, 15]}]

In [14]:
data['jobs'][8]

{'name': 'Job9',
 'gain': 25,
 'due_date': 18,
 'daily_penalty': 3,
 'working_days_per_qualification': {'E': 4, 'G': 3, 'H': 2}}

## Model and variable

In [15]:
# Instanciation du modèle
m = Model("Simple PL modelling")

# Création de variables
with open('/Users/cha/Desktop/Code/optimization-project/data/small.json') as f:
    data = json.load(f)

S = data['staff']
J = data['jobs']
Q = data['qualifications']
H = data['horizon']

X = m.addMVar(shape=(len(S), len(J), len(Q), H), vtype=GRB.BINARY)
Y = m.addMVar(shape=len(J), vtype=GRB.BINARY)
L = m.addMVar(shape=len(J))
E = m.addMVar(shape=len(J))
# maj du modèle
m.update()

## Defining constraints and objective function

In [16]:
def c1(x, i, t):
    res = 0
    for j in range(len(J)):
        for k in range(len(Q)):
            res += x[i, j, k, t]
    return res

def c4(x, j, k):
    res = 0
    for i in range(len(S)):
        for t in range(H):
            res += x[i, j, k, t]
    return res


# C1
for i in range(len(S)):
    for t in range(H):
        m.addConstr(c1(X, i, t) <= 1)

# C2
for i in range(len(S)):
    person = S[i]
    for t in person['vacations']:
        m.addConstr(c1(X, i, t) == 0)

# C3
for i in range(len(S)):
    person = S[i]
    for j in range(len(J)):
        job = J[j]
        for k in range(len(Q)):
            qualification = Q[k]
            if qualification not in person['qualifications'] and qualification not in list(job['working_days_per_qualification'].keys()):
                for t in range(H):
                    m.addConstr(X[i, j, k, t] == 0)

# C4
for j in range(len(J)):
    job = J[j]
    for k in range(len(Q)):
        qualification = Q[k]
        qualifications = list(job['working_days_per_qualification'].keys())
        if qualification in qualifications:
            njk = job['working_days_per_qualification'][qualification]
            m.addConstr(Y[j] * njk <= c4(X, j, k))

# C5
for j in range(len(J)):
    job = J[j]
    for k in range(len(Q)):
        qualification = Q[k]
        qualifications = list(job['working_days_per_qualification'].keys())
        if qualification in qualifications:
            njk = job['working_days_per_qualification'][qualification]
            m.addConstr(c4(X, j, k) <= njk)

# C6
for i in range(len(S)):
    for j in range(len(J)):
        for k in range(len(Q)):
            for t in range(H):
                m.addConstr(X[i, j, k, t] * t <= E[j])

# C7
for j in range(len(J)):
    job = J[j]
    dj = job["due_date"]
    m.addConstr(E[j] - dj <= L[j])

# Ej <= H
for j in range(len(J)):
    m.addConstr(E[j] <= H)

# Lj >= 0
for j in range(len(J)):
    m.addConstr(L[j] >= 0)

# Ej > 0
for j in range(len(J)):
    m.addConstr(E[j] >= 1)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


First objective function

In [17]:
def obj1(Y, L):
    res = 0
    for j in range(len(J)):
        job = J[j]
        gj = job["gain"]
        cj = job["daily_penalty"]
        res += (Y[j] * gj - L[j] * cj)
    return res

m.setObjective(
    obj1(Y, L),
    GRB.MAXIMIZE
)

Minimiser le nombre de projets sur lesquels un quelconque collaborateur est affecté

si x ≥ x0, imposer la contrainte x − x0 < M.δ (avec M un majorant de x) implique que δ = 1

In [18]:
# C'est un => qu'il faut introduire : si le nombre de jour sur lequel la personne travaille est supérieur à 1 alors la variable P[i, j] vaut 1

def working_on_project(X, i, j):
    res = 0
    for k in range(len(Q)):
        for t in range(H):
            res += X[i, j, k, t]
    return res

P = m.addMVar(shape=(len(S), len(J)), vtype=GRB.BINARY)

# Contrainte : nombre de projets par personne
for i in range(len(S)):
    for j in range(len(J)):
        res = working_on_project(X, i, j)
        m.addConstr(res == 0 or res - 1 <= H * len(Q) * P[i, j])

def obj2(P, i):
    res = 0
    for j in range(len(J)):
        res += P[i, j]
    return res


for i in range(len(S)):
    m.setObjective(
        obj2(P, i),
        GRB.MINIMIZE
    )

GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

Executer le projet le plus long en un minimum de jours

In [ ]:
M = m.addMVar(shape=(len(J), H), vtype=GRB.BINARY)

# Contrainte : nombre de projets par personne
for j in range(len(J)):
    for t in range(H):
        m.addConstr(M[j, t] <= 1)

def obj3(M, j):
    res = 0
    for t in range(H):
        res += M[j, t]
    return res

# for j in range(len(J)):
#     m.setObjective(
#         obj3(M, j),
#         GRB.MINIMIZE
#     )

: 

## Optimizing the model

In [ ]:
# Paramétrage (mode mute)
m.params.outputflag = 0
# Résolution du PL
m.optimize()
print("X: ", X.X, "\n")
print("Y: ", Y.X, "\n")
print("L: ", L.X, "\n")
print("E: ", E.X, "\n")
print("objective: ", m.objVal)

: 

In [ ]:
matE = E.X
matX = X.X
matY = Y.X
matL = L.X

: 

In [ ]:
import matplotlib.pyplot as plt

person0 = matX[0]
plt.table(
    cellText=person0[0],
    rowLabels=["A", "B", "C"],
    colLabels=[k for k in range(H)],
)
fig, ax = plt.subplots()

# hide axes
fig.patch.set_visible(False)
ax.axis('off')
ax.axis('tight')

ax.table(cellText=person0[0], colLabels=[k for k in range(H)], rowLabels=["A", "B", "C"], loc='center')

: 

In [ ]:
def rendu_tot():
    for j in range(len(J)):
        if matE[j] < J[j]["due_date"]:
            print("Yes")
rendu_tot()

: 

In [ ]:
for i in range(len(S)):
    for t in range(H):
        res = 0
        for j in range(len(J)):
            for k in range(len(Q)):
                res += matX[i, j, k, t]
        print(res)

: 

## Tuto internet

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

: 

In [ ]:
# Create a new model
m = gp.Model("matrix1")

# Create variables
x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x")

# Set objective
obj = np.array([1.0, 1.0, 2.0])
m.setObjective(obj @ x, GRB.MAXIMIZE)

# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# Build rhs vector
rhs = np.array([4.0, -1.0])

# Add constraints
m.addConstr(A @ x <= rhs, name="c")

# Optimize model
m.optimize()

print("here", x.X)
print('Obj: %g' % m.ObjVal)

: 